# Image processing for road marking

In [15]:
import cv2
import os
import pickle
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt 

def extract_lanes (img, lightness_th = 210, hue_th = (20, 30), sobelx_th=(30, 100)): 
    
    # Convert to HLS color space and separate the S and H channel 
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    h_channel = hls[:,:,0]
    s_channel = hls[:,:,2]
    
    # white color has a very high saturation
    binary_l = np.zeros_like(l_channel)
    binary_l[(l_channel >= lightness_th)] = 1
    
    # yellow color has a H between 20 and 30 
    binary_h = np.zeros_like(h_channel)
    binary_h[(h_channel <= hue_th[1]) & (h_channel >= hue_th[0])] = 1   #30-20
    
    # checking for gradient in x and y in the saturation channel  
    sobel_x = cv2.Sobel(s_channel, cv2.CV_64F, 1, 0, 5) # x direction
    sobel_y = cv2.Sobel(s_channel, cv2.CV_64F, 0, 1, 5) # y direction 
    
    # scalating the gradients
    abs_sobel_x = np.absolute(sobel_x)
    scaled_sobel_x = np.uint8(255*abs_sobel_x/np.max(abs_sobel_x))   
    abs_sobel_y = np.absolute(sobel_y)
    scaled_sobel_y = np.uint8(255*abs_sobel_y/np.max(abs_sobel_y))
    
    #
    binary_x = np.zeros_like(scaled_sobel_x)
    binary_x[(scaled_sobel_x > sobelx_th[0]) & (scaled_sobel_x < sobelx_th[1])] = 1
        

    # Combine the two binary thresholds
    combined_binary = np.zeros_like(binary_l)
    combined_binary[(binary_l == 1) | (binary_h == 1) | (binary_x ==1)] = 1
    
    
    return combined_binary



def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    ym_per_pix = 35/700 # meters per pixel in y dimension
    xm_per_pix = 3.7/660 # meters per pixel in x dimension
    
    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    left_fit_rw = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_rw = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
    
    # measuring the curvature
    y_eval = np.max(ploty)*ym_per_pix
    left_curverad = ((1 + (2*left_fit_rw[0]*y_eval+left_fit_rw[1])**2)**(3/2))/(2*left_fit_rw[0])  
    right_curverad = ((1 + (2*right_fit_rw[0]*y_eval+right_fit_rw[1])**2)**(3/2))/(2*right_fit_rw[0]) 
    
    # measuring how centered the vehicle is 
    mploty = np.max(ploty)
    left_x = left_fit[0]*mploty**2 + left_fit[1]*mploty + left_fit[2]
    right_x = right_fit[0]*mploty**2 + right_fit[1]*mploty + right_fit[2]
    mid_point = 0.5 * left_x + 0.5 * right_x
    deviation = (640 -mid_point) * xm_per_pix
            
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    
    # Colors in the left and right lane regions
    color_warp[lefty, leftx] = [255, 0, 0]
    color_warp[righty, rightx] = [0, 0, 255]
    
    # Plots the left and right polynomials on the lane lines
    color_warp[ploty.astype(int), left_fitx.astype(int)]  = [255, 255, 0]
    color_warp[ploty.astype(int), right_fitx.astype(int)] = [255, 255, 0]
    

    return color_warp, left_curverad, right_curverad, deviation
    

# Reading all the images
images_list = os.listdir("test_images/")

# Reading the saved camera matrix and distortion coefficients
cam_pickle = pickle.load( open( "camera_parameters.pickle", "rb") ) 
mtx = cam_pickle[0]
dist = cam_pickle[1]

# Reading perspective transformation
M_pickle = pickle.load( open( "p_transformation.pickle", "rb") ) 
M = M_pickle[0]
Minv = M_pickle[1]


for fname in images_list:
    
    #reading each image
    img = mpimg.imread('test_images/' + fname)

    #correcting distortion 
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    # saving the final image
    mpimg.imsave('output_images/udst_' + fname, dst)

    #extracting lanes
    lanes = extract_lanes(dst)
    mpimg.imsave('output_images/lines_' + fname, lanes, cmap = plt.cm.gray)
    
    #applying perspective transformation
    bird_eye = cv2.warpPerspective(lanes, M,(img.shape[1], img.shape[0]), flags = cv2.INTER_LINEAR)   
    mpimg.imsave('output_images/bev_' + fname, bird_eye)
    
    #obtaining the image with the lanes
    color_warp, left_curverad, right_curverad, deviation = fit_polynomial(bird_eye)
    mpimg.imsave('output_images/warp_' + fname, color_warp)
    
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    
    # Combine the result with the original image
    result = cv2.addWeighted(dst, 1, newwarp, 0.3, 0)
    
    # mean of the curvature
    curvature = np.mean ([np.abs(left_curverad), np.abs(right_curverad)])
    
    # direction left/right
    direction = ''
    if deviation > 0 : 
        direction = 'left'
    else :
        direction = 'right'
    
    # printing on top of the image
    cv2.putText(result,'Radius of Curvature = {:.1f}(m)'.format(curvature),(30,50), cv2.FONT_HERSHEY_SIMPLEX, 2,(255,255,255),2)
    cv2.putText(result,'Vehicle is {:.3f} {} of the center'.format(deviation, direction),(30,100), cv2.FONT_HERSHEY_SIMPLEX, 2,(255,255,255),2)
    
    # saving the final image
    mpimg.imsave('output_images/out_' + fname, result)
